In [38]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *


from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

In [2]:
desde = '2022-04-13'
hasta = '2022-07-13'
city = 'BOG'

In [3]:
query = """
WITH 
count_fulfillments_pasados AS (
	SELECT 
	    COUNT(DISTINCT boi.order_item_id) AS count_items,
	    bfg.order_id
     
	FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"              bfg
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order"                     bo      ON bo.order_id = bfg.order_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"          ffg     ON bfg.fulfillment_group_id = ffg.fulfillment_group_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"         bfo     ON bfg.fulfillment_group_id = bfo.fulfillment_group_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"             bop     ON bop.order_id = bo.order_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order"                      fo      ON fo.order_id = bo.order_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON bo.site_disc = s.site_id
	LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                fot     ON fot.fb_order_type_id=fo.fb_order_type_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"   bfgi    ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON bfgi.order_item_id= boi.order_item_id
	
	WHERE      
        ffg.close_date between '{desde}' AND '{hasta}'
        --AND s.site_identifier_value = {city}
        AND bo.order_status='SUBMITTED'
        AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
        AND fo.fb_order_status_id IN (1,6,7,8)
        AND bop.archived = 'N'
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
	GROUP BY 2
),

ful AS (

    SELECT DISTINCT
        bo.order_id,
        ffg.close_date
        
    FROM postgres_broadleaf_federate."broadleaf.blc_order"                      bo
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg     ON bfg.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo     ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo      ON fo.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop     ON bop.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s       ON s.site_id = bo.site_disc
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot     ON fot.fb_order_type_id=fo.fb_order_type_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment"     ba      ON ba.order_id = bo.order_id
    
    WHERE fo.fb_order_status_id IN (1,6,7,8)
        AND ffg.close_date between '{desde}' AND '{hasta}'
        --AND s.site_identifier_value = {city}
        AND bo.order_status = 'SUBMITTED'
        AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
        AND bop.archived = 'N'
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
        
    GROUP BY 1,2
)

SELECT DISTINCT
    --bo.order_id,
    bo.customer_id,
    s.site_identifier_value as region_code,
    --TO_CHAR(ful.close_date, 'YYYY-mm-dd') AS day,
    bcat2.name as cat,
    bcat.name as subcat,
    --bs.sku_id,
    bs.addl_product_id AS padre_sku_id,
    bs.name AS product,
    SUM(boi.quantity) AS quantity,
    CASE 
        WHEN s.site_identifier_value = 'CMX' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'GDL' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'PBC' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'MTY' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'SPO' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'BHZ' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'CWB' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'VCP' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
    ELSE ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3000) END AS net_gmv_usd
    
FROM 
    postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"                  bfgi
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"            bfg     ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"             ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                        bo      ON bo.order_id = bfg.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON bo.site_disc = s.site_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"                   boi     ON bfgi.order_item_id= boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                    foi     ON boi.order_item_id= foi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                         fow     ON fow.owner_id = ffg.owner_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                         fo      ON fo.order_id = bo.order_id
    INNER JOIN ful                                                                              ON ful.order_id = bo.order_id
    LEFT JOIN count_fulfillments_pasados                                                cf      ON bfg.order_id = cf.order_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                    fot     ON fot.fb_order_type_id=fo.fb_order_type_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"          bdoi    ON bdoi.order_item_id = boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bdoi.sku_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                 bcx     ON bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and bcx.default_reference = 'true'
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"                      bcat    ON bp.default_category_id = bcat.category_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"                      bcat2   ON bcx.category_id = bcat2.category_id
    --LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"          boipd   ON boipd.order_item_id=boi.order_item_id
    --LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"            boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id
WHERE 
    ffg.close_date between '{desde}' AND '{hasta}'
    --AND s.site_identifier_value = {city}
    AND bo.order_status='SUBMITTED'
    AND fo.fb_order_status_id IN (1,6,7,8)
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND bcat2.name IN ('Abarrotes','Aseo e Higiene','Bebidas','Congelados','Desechables','Lácteos & Huevos','Carne, Pollo & Pescados', 'Frutas & Verduras', 'Bebidas','Mercearia','Limpeza e Higiene','Laticínios e Ovos','Frutas e Verduras','Congelados','Descartáveis','Carnes, Aves e Peixes')

GROUP BY 1,2,3,4,5,6--,7,8--,9
""".format(desde=desde, hasta=hasta, city=city)
df = read_connection_data_warehouse.runQuery(query)

In [4]:
df

,customer_id,region_code,cat,subcat,padre_sku_id,product,quantity,net_gmv_usd
0,64837818,CMX,Frutas & Verduras,Verduras,-201510,Tomate Verde 2a,10,5.320479416362688900468900
1,147470811,CWB,Limpeza e Higiene,Utensílios de Limpeza,285002,Esponja multiuso Brilhex leve 12 pague 10,3,6.146907216494845360824700
2,38053545,BAQ,Aseo e Higiene,"Detergente, Jabón & Lavalozas",130864,Detergente en polvo Ultrex x 1000g,2,4.031720000000000000000000
3,-263376,CMX,"Carne, Pollo & Pescados",Cerdo Fresco,190010,Pierna de Cerdo Sin Hueso Limpia,2,62.461698801459093277748800
4,161337892,MDE,Abarrotes,Sal & Sazonadores,293160,Sal Refisal arroba 25 unid de 500gr,2,15.079333333333333333333333
...,...,...,...,...,...,...,...,...
2807604,58008766,SPO,Mercearia,"Azeites, Óleos e Vinagres",74424,Óleo Concordia 900 ml,20,48.247422680412371134020600
2807605,17417664,CMX,Aseo e Higiene,"Detergente, Jabón & Lavatrastes",96804,Limpiador Pino Los Patitos 1L,4,3.435643564356435643564300
2807606,15445630,BAQ,Aseo e Higiene,"Detergente, Jabón & Lavalozas",260669,Lavaloza en crema limón Dessin x 500 gr,3,2.528156666666666666666666
2807607,53679349,BOG,Aseo e Higiene,Desinfectantes & Sanitizantes,261010,Limpiador desinfectante Lavanda PQP Profesiona...,1,3.843333333333333333333333


In [5]:
df['product'] = df['product'].str.strip()
df.dropna(axis=0, subset=['customer_id'], inplace=True)
df['customer_id'] = df['customer_id'].astype('str')

In [6]:
basket = (df[df['region_code'] =="BOG"]
          .groupby(['customer_id', 'product'])['quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('customer_id'))

In [7]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [8]:
basket_sets

product,Aceite Kawsay de oliva extra virgen 450 ml,Aceite Kawsay de oliva extra virgen 900 ml,Aceite Naoli 20LT,Aceite Naoli 20LT (Sin IVA),Aceite Naoli 3LT,Aceite Oleosuper x 20 Lts desde 1 und,Aceite Oleosuper x 20 Lts desde 2 unds,Aceite Riquísimo 1LT,Aceite Riquísimo 1LT (Sin IVA),Aceite Sólido Bucaro,...,apio atado tamaño hogar,apio manojo,brocoli de a 4,brocoli de a libra,brocoli docena,espinaca,espinaca Bogotana,lechuga crespa verde,pollo entero congelado sin visceras,Ñame
customer_id,,,,,,,,,,,,,,,,,,,,,
-200013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0
-200025,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
-200037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
-200050,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
-200051,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99949310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99952107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99952792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
frequent_itemsets = fpgrowth(basket_sets, min_support=0.05, use_colnames=True)
frequent_itemsets

/home/jupyter-felipe.guevara/.local/lib/python3.7/site-packages/mlxtend/frequent_patterns/fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


,support,itemsets
0,0.228519,(Limón Tahití)
1,0.138648,(Cilantro Patirojo)
2,0.134520,(Pepino Cohombro)
3,0.106884,(lechuga crespa verde)
4,0.098485,(Ajo)
...,...,...
216,0.056708,"(Servilleta Popular x 330 un, Vaso transparent..."
217,0.058108,"(Servilleta Popular x 330 un, Zanahoria)"
218,0.051683,"(Azúcar en bulto 50kg, Harina de trigo El Lobo)"
219,0.053873,"(Servilleta Popular x 150 un, Zanahoria)"


In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [37]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1) 
rules
#support how many times of be together (P(AuC)) 
#confidence is equal to P(C/A)
#lift is to evaluate the support and confidence rull, there are times when both of these measures may be high, and yet still produce a rule that is not useful -> (Rule Support) /(Support(Antecedent) * Support(Consequent))
"""
Any rule with an improvement of less than 1 does not indicate a real cross-selling opportunity,
no matter how high its support and confidence, because it actually offers less ability to predict a purchase than does random chance.
"""

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Amarre Huevos Rojos AA),(Aceite Naoli 3LT),0.195069,0.252746,0.068301,0.350138,1.385337,0.018998,1.149866
1,(Aceite Naoli 3LT),(Amarre Huevos Rojos AA),0.252746,0.195069,0.068301,0.270236,1.385337,0.018998,1.103002
2,(Arroz Naoli 500gx25un),(Aceite Naoli 3LT),0.139581,0.252746,0.066327,0.475186,1.880097,0.031048,1.423847
3,(Aceite Naoli 3LT),(Arroz Naoli 500gx25un),0.252746,0.139581,0.066327,0.262425,1.880097,0.031048,1.166552
4,(Arroz Roa 500gx25un),(Aceite Naoli 3LT),0.190080,0.252746,0.076125,0.400491,1.584561,0.028083,1.246444
...,...,...,...,...,...,...,...,...,...
381,(Cebolla Cabezona Blanca Semi Mediana por kilo...,"(Limón Tahití, Zanahoria)",0.120128,0.129459,0.051217,0.426352,3.293324,0.035665,1.517552
382,(Limón Tahití),"(Zanahoria, Cebolla Cabezona Blanca Semi Media...",0.228519,0.078817,0.051217,0.224124,2.843604,0.033206,1.187282
383,(Zanahoria),"(Limón Tahití, Cebolla Cabezona Blanca Semi Me...",0.315878,0.067332,0.051217,0.162141,2.408083,0.029948,1.113156
384,(Cebolla Cabezona Blanca Semi Mediana por kilo),"(Limón Tahití, Zanahoria, Cebolla Larga Junca)",0.250485,0.076699,0.051217,0.204471,2.665867,0.032005,1.160611


In [46]:
rules.sort_values(by = 'lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
221,(Zuquini Verde),(Zuquini Amarillo),0.076233,0.069162,0.051073,0.669962,9.686814,0.045801,2.820398
220,(Zuquini Amarillo),(Zuquini Verde),0.069162,0.076233,0.051073,0.738454,9.686814,0.045801,3.531943
191,(espinaca Bogotana),(Pepino Cohombro),0.097086,0.134520,0.053765,0.553789,4.116776,0.040705,1.939621
190,(Pepino Cohombro),(espinaca Bogotana),0.134520,0.097086,0.053765,0.399680,4.116776,0.040705,1.504055
319,"(Limón Tahití, Cebolla Larga Junca)",(Pepino Cohombro),0.109360,0.134520,0.052868,0.483426,3.593710,0.038157,1.675424
...,...,...,...,...,...,...,...,...,...
16,(Limón Tahití),(Aceite Naoli 3LT),0.228519,0.252746,0.064891,0.283964,1.123518,0.007134,1.043599
173,(Vaso transparente 7 oz WAU x 50 un),(Limón Tahití),0.207487,0.228519,0.051037,0.245978,1.076401,0.003623,1.023155
172,(Limón Tahití),(Vaso transparente 7 oz WAU x 50 un),0.228519,0.207487,0.051037,0.223339,1.076401,0.003623,1.020411
46,(Amarre Huevos Rojos AA),(Zanahoria),0.195069,0.315878,0.064604,0.331187,1.048463,0.002986,1.022889
